# Database data validation

In this notebook, we will see how we can use the great_expectations package to validate data in our database.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NatanMish/data_validation/blob/main/notebooks/1_database_data_validation.ipynb)

#### Install the required packages and import them to the notebook


In [1]:
!pip install -U great_expectations pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 5.3 MB/s 
     |████████████████████████████████| 139 kB 67.2 MB/s 
     |████████████████████████████████| 793 kB 63.8 MB/s 
     |████████████████████████████████| 4.1 MB 27.5 MB/s 
     |████████████████████████████████| 9.9 MB 44.2 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
     |████████████████████████████████| 109 kB 45.9 MB/s 
     |████████████████████████████████| 1.6 MB 48.4 MB/s 
     |████████████████████████████████| 59 kB 1.9 MB/s 
     |████████████████████████████████| 423 kB 50.8 MB/s 
     |████████████████████████████████| 127 kB 60.4 MB/s 
     |████████████████████████████████| 546 kB 49.2 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.

In [2]:
# import the required packages
import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.profile.user_configurable_profiler import (
    UserConfigurableProfiler,
)
from great_expectations.checkpoint import SimpleCheckpoint
from ruamel import yaml
import pandas as pd

[![Great Expectations](https://docs.greatexpectations.io/img/great-expectations-long-logo.svg)]

Named after the famous 19th century novel, Great Expectations is a shared, open sourced package for data quality. It helps eliminate pipeline debt, through data testing, documentation, and profiling. It is a tool for data scientists, data engineers, and data analysts to validate data. GE has many useful integrations and can be connected directly to SQL databases, Apache Spark, Apache Airflow, Bigquery, and more. In this tutorial, we will validate a database hosted on a  local file system, but the process for a cloud file system such as a Data Lake, Azure Blob Storage, GCP bucket or AWS S3 is almost identical.

**Terminology**
1. *Data Context* - The primary entry point for a Great Expectations deployment, with configurations and methods for all supporting components.

2. *Data Source* - Provides a standard API for accessing and interacting with data from a wide variety of source systems.

3. *Data Asset* - A collection of records within a Datasource which is usually named based on the underlying data system and sliced to correspond to a desired specification.

4. *Expectation Suite* - A collection of verifiable assertions about data.

5. *Validation* - The act of applying an Expectation Suite to a Batch.

6. *Batch Identifier* - contains information that uniquely identifies a specific batch from the Data Asset, such as the delivery date or query time.

7. *Data Connector* - Provides the configuration details based on the source data system which are needed by a Datasource to define Data Assets.

### 1. Create a Data Context

We will now create a data context, which is the first step in setting up Great Expectations for our project. Creating a data context is actually most easily done in bash using the great_expectations CLI. Run the shell command below and this will initialize a new data context in the current directory. The `echo y` bit is used to suppress the interactive prompt. You will now see a new directory called `great_expectations` created in your current directory.

In [3]:
!echo y | great_expectations init

Using v3 (Batch Request) API

  ___              _     ___                  _        _   _
 / __|_ _ ___ __ _| |_  | __|_ ___ __  ___ __| |_ __ _| |_(_)___ _ _  ___
| (_ | '_/ -_) _` |  _| | _|\ \ / '_ \/ -_) _|  _/ _` |  _| / _ \ ' \(_-<
 \___|_| \___\__,_|\__| |___/_\_\ .__/\___\__|\__\__,_|\__|_\___/_||_/__/
                                |_|
             ~ Always know what to expect from your data ~

Let's create a new Data Context to hold your project configuration.

Great Expectations will create a new directory with the following structure:

    great_expectations
    |-- great_expectations.yml
    |-- expectations
    |-- checkpoints
    |-- plugins
    |-- .gitignore
    |-- uncommitted
        |-- config_variables.yml
        |-- data_docs
        |-- validations

OK to proceed? [Y/n]: 

Congratulations! You are now ready to customize your Great Expectations configuration.

You can customize your configuration in many ways. Here are some examples:

  Use the CLI to:
    - Ru

After running the init command, your great_expectations directory will contain all the important components of a local Great Expectations deployment. This is what the directory structure looks like:

- `great_expectations.yml` contains the main configuration of your deployment.
The expectations directory stores all your Expectations as JSON files. If you want to store them somewhere else, you can change that later.

- The `plugins/` directory holds code for any custom plugins you develop as part of your deployment.

- The `uncommitted/` directory contains files that shouldn’t live in version control. It has a .gitignore configured to exclude all its contents from version control. The main contents of the directory are:
    1. `uncommitted/config_variables.yml`, which holds sensitive information, such as database credentials and other secrets.
    2. `uncommitted/data_docs`, which contains Data Docs generated from Expectations, Validation Results, and other metadata.
    3. `uncommitted/validations`, which holds Validation Results generated by Great Expectations.

<div>
<img src="https://docs.greatexpectations.io/assets/images/data_context_does_for_you-df2eca32d0152ead16cccd5d3d226abb.png" width="1000"/>
</div>

### 2. Create a Data Source

In [4]:
# We will start by reading in the GE data context we have created in the previous step
context = ge.get_context()

Now we will script a yaml file to create a data source. We will need the following configuration parameters:

In [5]:
datasource_name = "house_prices"
# Data Source - Provides a standard API for accessing and interacting with data from a wide variety of source systems.

In [6]:
execution_engine = "PandasExecutionEngine"  # alternatively we can use SparkExecutionEngine for PySpark oriented
# projects or SqlAlchemyExecutionEngine for creating a SQL database data source.
data_directory = "data"

In [7]:
data_asset_name = f"{datasource_name}_survey_2006"
# Data Asset - A collection of records within a Datasource which is usually named based on the underlying data system and sliced to correspond to a desired specification.

In [8]:
runtime_data_connector_name = "runtime_batch_files_connector"
# Data Connector - Provides the configuration details based on the source data system which are needed by a Datasource to define Data Assets.

In [9]:
batch_identifier_name = "pipeline_step"
# Batch Identifier - contains information that uniquely identifies a specific batch from the Data Asset, such as the delivery date or query time.

In [10]:
datasource_config = {
    "name": datasource_name,
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": execution_engine,
    },
    "data_connectors": {
        runtime_data_connector_name: {
            "class_name": "RuntimeDataConnector",
            "module_name": "great_expectations.datasource.data_connector",
            "assets": {
              data_asset_name: {
                "class_name": "Asset",
                "batch_identifiers": [batch_identifier_name],
                "module_name": "great_expectations.datasource.data_connector.asset"}}
        },
    },
}

In [11]:
# Test that the configuration is valid
context.test_yaml_config(yaml.dump(datasource_config))

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	runtime_batch_files_connector:RuntimeDataConnector

	runtime_batch_files_connector : RuntimeDataConnector

	Available data_asset_names (1 of 1):
		house_prices_survey_2006 (0 of 0): []

	Unmatched data_references (0 of 0):[]



In [12]:
# If the configuration is valid, we can create the datasource
context.add_datasource(**datasource_config)

In [24]:
# Now we can see that the datasource was created.
context.list_datasources()

[{'class_name': 'Datasource',
  'data_connectors': {'runtime_batch_files_connector': {'class_name': 'RuntimeDataConnector',
    'assets': {'house_prices_survey_2006': {'batch_identifiers': ['pipeline_step'],
      'class_name': 'Asset',
      'module_name': 'great_expectations.datasource.data_connector.asset'}},
    'module_name': 'great_expectations.datasource.data_connector'}},
  'execution_engine': {'class_name': 'PandasExecutionEngine',
   'module_name': 'great_expectations.execution_engine'},
  'module_name': 'great_expectations.datasource',
  'name': 'house_prices'}]

### 3. Create an Expectation Suite
Expectations are the core of Great Expectations. They are the assertions that are used to validate data. Let's create an expectation suite which is a collection of expectations. This diagram below shows how we can define good expectations for our data.

<div>
<img src="https://docs.greatexpectations.io/assets/images/where_expectations_come_from-b3504cf51ad304c8e4a73677a0e73156.png" width="1000"/>
</div>

We will create expectations while exploring the data in the notebook. The method below behaves  exactly the same as `pandas.read_csv`. Similarly wrapped versions of other pandas methods (`read_excel`, `read_table`, `read_parquet`, `read_pickle`, `read_json`, etc.) are also available.

In [85]:
house_data = ge.read_csv("https://github.com/NatanMish/data_validation/blob/a77b247b25c6622ce0c8f8cbc505228161c31a3c/data/train.csv?raw=true")

In [86]:
# The house_data variable is a pandas dataframe with all the methods and properties we know and love. We can use the `head` method to see the
# first few rows of the data.
house_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [88]:
# beyond the Pandas methods and properties, we can use GE's expectations methods to define expectations. 
# In Jupyter, type in `house_data.expect` and press tab to see the list of available expectations.
dir(house_data)

['Alley',
 'BedroomAbvGr',
 'BldgType',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtQual',
 'BsmtUnfSF',
 'CentralAir',
 'Condition1',
 'Condition2',
 'Electrical',
 'EnclosedPorch',
 'ExterCond',
 'ExterQual',
 'Exterior1st',
 'Exterior2nd',
 'Fence',
 'FireplaceQu',
 'Fireplaces',
 'Foundation',
 'FullBath',
 'Functional',
 'GarageArea',
 'GarageCars',
 'GarageCond',
 'GarageFinish',
 'GarageQual',
 'GarageType',
 'GarageYrBlt',
 'GrLivArea',
 'HalfBath',
 'Heating',
 'HeatingQC',
 'HouseStyle',
 'Id',
 'KitchenAbvGr',
 'KitchenQual',
 'LandContour',
 'LandSlope',
 'LotArea',
 'LotConfig',
 'LotFrontage',
 'LotShape',
 'LowQualFinSF',
 'MSSubClass',
 'MSZoning',
 'MasVnrArea',
 'MasVnrType',
 'MiscFeature',
 'MiscVal',
 'MoSold',
 'Neighborhood',
 'OpenPorchSF',
 'OverallCond',
 'OverallQual',
 'PavedDrive',
 'PoolArea',
 'PoolQC',
 'RoofMatl',
 'RoofStyle',
 'SaleCondition',
 'SalePrice',
 'Sal

In [89]:
# Let's create a few example expectations and see if they are valid on this dataset.
house_data.expect_column_to_exist("Id")

{
  "meta": {},
  "expectation_config": {
    "kwargs": {
      "column": "Id",
      "result_format": "BASIC"
    },
    "meta": {},
    "expectation_type": "expect_column_to_exist"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {},
  "success": true
}

In [90]:
house_data.expect_column_to_exist("Id").success

True

Notice the `"success": true` key in the result dictionary, this means the expectation is valid for this data source

In [91]:
house_data.expect_column_values_to_be_unique("Id")

{
  "meta": {},
  "expectation_config": {
    "kwargs": {
      "column": "Id",
      "result_format": "BASIC"
    },
    "meta": {},
    "expectation_type": "expect_column_values_to_be_unique"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 1460,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

The expectation above checked the contents of the column, hence we got a few other useful metrics, showing how many 
rows were inspected, how many were missing etc.

In [92]:
# This expectation should fail, lets see what happens:
house_data.expect_column_max_to_be_between("SalePrice", 0, 100000)

{
  "meta": {},
  "expectation_config": {
    "kwargs": {
      "column": "SalePrice",
      "min_value": 0,
      "max_value": 100000,
      "result_format": "BASIC"
    },
    "meta": {},
    "expectation_type": "expect_column_max_to_be_between"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": 755000,
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "success": false
}

The returned dictionary shows that the expectation is not valid, and the value observed that is not in the expected range.
Here are a few more useful expectation definitions:

In [93]:
house_data.expect_column_distinct_values_to_be_in_set("MSZoning", ["C (all)", "FV", "RH", "RL", "RM"])

{
  "meta": {},
  "expectation_config": {
    "kwargs": {
      "column": "MSZoning",
      "value_set": [
        "C (all)",
        "FV",
        "RH",
        "RL",
        "RM"
      ],
      "result_format": "BASIC"
    },
    "meta": {},
    "expectation_type": "expect_column_distinct_values_to_be_in_set"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": [
      "C (all)",
      "FV",
      "RH",
      "RL",
      "RM"
    ],
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "success": true
}

In [94]:
house_data.expect_column_mean_to_be_between("GrLivArea", 0, 10000)

{
  "meta": {},
  "expectation_config": {
    "kwargs": {
      "column": "GrLivArea",
      "min_value": 0,
      "max_value": 10000,
      "result_format": "BASIC"
    },
    "meta": {},
    "expectation_type": "expect_column_mean_to_be_between"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": 1515.463698630137,
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "success": true
}

In [95]:
# This will create an expectation suite from all the valid expectations we created above.
house_data.get_expectation_suite()
# If we want the non-valid expectations as well, we can use the `get_expectation_suite` method with the 
# `discard_failed_expectations` parameter set to True. If there are any duplicate expectations in the suite, 
# the duplicates will be discarded:
# house_data.get_expectation_suite(discard_failed_expectations=False)

{
  "ge_cloud_id": null,
  "expectations": [
    {
      "kwargs": {
        "column": "Id"
      },
      "meta": {},
      "expectation_type": "expect_column_to_exist"
    },
    {
      "kwargs": {
        "column": "Id"
      },
      "meta": {},
      "expectation_type": "expect_column_values_to_be_unique"
    },
    {
      "kwargs": {
        "column": "MSZoning",
        "value_set": [
          "C (all)",
          "FV",
          "RH",
          "RL",
          "RM"
        ]
      },
      "meta": {},
      "expectation_type": "expect_column_distinct_values_to_be_in_set"
    },
    {
      "kwargs": {
        "column": "GrLivArea",
        "min_value": 0,
        "max_value": 10000
      },
      "meta": {},
      "expectation_type": "expect_column_mean_to_be_between"
    }
  ],
  "meta": {
    "great_expectations_version": "0.15.18"
  },
  "expectation_suite_name": "default",
  "data_asset_type": "Dataset"
}

In [96]:
# This line will save the expectation suite to the data context
expectation_suite_name = "my_expectations"
context.save_expectation_suite(house_data.get_expectation_suite(), expectation_suite_name)

### Exercise 1
Check the following expectations to see if they are valid on the house_data dataframe:

(Not all the expectations were included in the examples above. You can find more expectations in the [expectations directory](https://greatexpectations.io/expectations).)
1. `Street` column should be a string.
2. `LandContour` column cannot be null.
3. `YearBuilt` minimal value should be between 1700 and 1900.
4. `LotArea` median value should be between 5000 and 15000.
5. The most common values in `SaleType` must be either `WD` or `New`.

In [63]:
# your answers here:
#house_data.expect_column_values_to_be_of_type("Street", 'str')
#house_data.expect_column_values_to_not_be_null('LandContour')
#house_data.expect_column_min_to_be_between("YearBuilt", 1700, 1900)
#house_data.expect_column_median_to_be_between("LotArea", 5000, 15000)
house_data.expect_column_most_common_value_to_be_in_set("SaleType", ['WD', 'New'])


{
  "meta": {},
  "expectation_config": {
    "kwargs": {
      "column": "SaleType",
      "value_set": [
        "WD",
        "New"
      ],
      "result_format": "BASIC"
    },
    "meta": {},
    "expectation_type": "expect_column_most_common_value_to_be_in_set"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": [
      "WD"
    ],
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "success": true
}

*Exercise solutions can be found in the exercise solutions file in the current directory.*

### 4. Validate the Data
We will now validate the test data using the expectations we have created for the train data.

<div>
<img src="https://docs.greatexpectations.io/assets/images/how_a_checkpoint_works-10e7fda2c9013d98a36c1d8526036764.png" width="1000"/>
</div>

In [97]:
checkpoint_name = "data_batch_appended"
# Checkpoint - The primary means for validating data in a production deployment of Great Expectations.

In [98]:
checkpoint_config = {
    "name": checkpoint_name,
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": {
                "datasource_name": datasource_name,
                "data_connector_name": runtime_data_connector_name,
                "data_asset_name": data_asset_name,
            },
            "expectation_suite_name": expectation_suite_name,
        }
    ],
}
context.add_checkpoint(**checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "data_batch_appended",
  "profilers": [],
  "runtime_configuration": {},
  "validations": [
    {
      "batch_request": {
        "datasource_name": "house_prices",
        "data_connector_name": "runtime_batch_files_connector",
        "data_asset_name": "house_prices_survey_2006"
      },
      "expectation_suite_name": "my_expectations"
    }
  ]
}

Looking at the dictionary returned by the `add_checkpoint` methods we can see what are the actions performed every time the checkpoint will run:
1. Store validation result.
2. Store evaluation parameters.
3. Update data docs. (we will look at the data docs later in this notebook)

In [99]:
house_data_test = ge.read_csv("https://github.com/NatanMish/data_validation/blob/a77b247b25c6622ce0c8f8cbc505228161c31a3c/data/test.csv?raw=true")

In [100]:
results = context.run_checkpoint(
    checkpoint_name=checkpoint_name,
    batch_request={
        "runtime_parameters": {"batch_data": house_data_test},
        "batch_identifiers": {
            batch_identifier_name: "step_1"
        },
    },
)

Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/jinja2/environment.py:1088: DeprecationWarning: 'soft_unicode' has been renamed to 'soft_str'. The old name will be removed in MarkupSafe 2.1.
  return concat(self.root_render_func(self.new_context(vars)))


In [ ]:
results

In [101]:
# Let's take a look at the validation result object we got:
run_identifier = next(iter(results['run_results']))
results['run_results'][run_identifier]['validation_result']['statistics']

{'evaluated_expectations': 4,
 'successful_expectations': 4,
 'unsuccessful_expectations': 0,
 'success_percent': 100.0}

In [107]:
# Here is an example of one of the validations on one of the expectations. The check has passed and there are some 
# useful extra details too.
results['run_results'][run_identifier]['validation_result']['results']

[{
   "meta": {},
   "expectation_config": {
     "kwargs": {
       "column": "Id",
       "batch_id": "3722fd4c2e47cabe7527b03f86860621"
     },
     "meta": {},
     "expectation_type": "expect_column_to_exist"
   },
   "exception_info": {
     "raised_exception": false,
     "exception_traceback": null,
     "exception_message": null
   },
   "result": {},
   "success": true
 }, {
   "meta": {},
   "expectation_config": {
     "kwargs": {
       "column": "Id",
       "batch_id": "3722fd4c2e47cabe7527b03f86860621"
     },
     "meta": {},
     "expectation_type": "expect_column_values_to_be_unique"
   },
   "exception_info": {
     "raised_exception": false,
     "exception_traceback": null,
     "exception_message": null
   },
   "result": {
     "element_count": 1459,
     "unexpected_count": 0,
     "unexpected_percent": 0.0,
     "partial_unexpected_list": [],
     "missing_count": 0,
     "missing_percent": 0.0,
     "unexpected_percent_total": 0.0,
     "unexpected_percent_no

#### How does an invalid data checkpoint look like?
Glad you asked, let's inject a duplicate value to our `Id` column to see how it behaves:

In [108]:
# This will create a duplicate id value for two separate records
house_data_test.at[0, 'Id'] = 1462

In [109]:
validator = {
            "batch_request": {
                "datasource_name": datasource_name,
                "data_connector_name": runtime_data_connector_name,
                "data_asset_name": data_asset_name,
            },
            "expectation_suite_name": expectation_suite_name,
        }

In [110]:
# Notice how we've adde the runtime configuration and result format = COMPLETE, this will give us the unexpected index list

bad_data_checkpoint_name = "my_bad_data_checkpoint"
bad_data_checkpoint_config = {
    "name": bad_data_checkpoint_name,
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "runtime_configuration": {
        "result_format": {
            "result_format": "COMPLETE", # BASIC
            "include_unexpected_rows": True
        }
    },
    "validations": [
      validator 
    ],
}
context.add_checkpoint(**bad_data_checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "my_bad_data_checkpoint",
  "profilers": [],
  "runtime_configuration": {
    "result_format": {
      "result_format": "COMPLETE",
      "include_unexpected_rows": true
    }
  },
  "validations": [
    {
      "batch_request": {
        "datasource_name": "house_prices",
        "data_connector_name": "runtime_batch_files_connector",
        "data_asset_name": "house_prices_survey_2006"
  

In [111]:
results_bad_data_checkpoint = context.run_checkpoint(
    checkpoint_name=bad_data_checkpoint_name,
    batch_request={
        "runtime_parameters": {"batch_data": house_data_test},
        "batch_identifiers": {
            batch_identifier_name: "step_2"
        },
    },
)

Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/jinja2/environment.py:1088: DeprecationWarning: 'soft_unicode' has been renamed to 'soft_str'. The old name will be removed in MarkupSafe 2.1.
  return concat(self.root_render_func(self.new_context(vars)))


In [112]:
# As expected, not all expectations were successful.
bad_data_run_identifier = next(iter(results_bad_data_checkpoint['run_results']))
results_bad_data_checkpoint['run_results'][bad_data_run_identifier]['validation_result']['statistics']

{'evaluated_expectations': 4,
 'successful_expectations': 3,
 'unsuccessful_expectations': 1,
 'success_percent': 75.0}

In [113]:
# And here is the summary for the failed expectation
results_bad_data_checkpoint['run_results'][bad_data_run_identifier]['validation_result']['results'][1]

{
  "meta": {},
  "expectation_config": {
    "kwargs": {
      "column": "Id",
      "batch_id": "8cfc8390f2477f0261bae2a270905154"
    },
    "meta": {},
    "expectation_type": "expect_column_values_to_be_unique"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 1459,
    "unexpected_count": 2,
    "unexpected_percent": 0.1370801919122687,
    "partial_unexpected_list": [
      1462,
      1462
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.1370801919122687,
    "unexpected_percent_nonmissing": 0.1370801919122687,
    "unexpected_rows": [
      {
        "Id": 1462,
        "MSSubClass": 20,
        "MSZoning": "RH",
        "LotFrontage": 80.0,
        "LotArea": 11622,
        "Street": "Pave",
        "Alley": null,
        "LotShape": "Reg",
        "LandContour": "Lvl",
        "Utilities": "AllPub",
        "LotConfig": "I

In [117]:
# We can unpack the unexpected data indices list
unexpected_data_indices = results_bad_data_checkpoint['run_results'][bad_data_run_identifier]['validation_result']['results'][1]['result']['unexpected_index_list']
unexpected_data_indices

[0, 1]

In [ ]:
# And then filter the invalid data from the dataframe
filtered_house_data_test = house_data_test[~house_data_test.index.isin(unexpected_data_indices)]

### 6. Create Data Docs

In [ ]:
# Build the data docs, in Jupyter a new tab will open up with the data docs page
!echo y | great_expectations docs build --site-name local_site

In [ ]:
# in Google Colab you'll have to download the folder and open it up locally in /uncommited/data_docs/local_site/index.html. 
# Uncomment and run the 3 lines below:
# from google.colab import files
# !zip -r data_docs.zip great_expectations/uncommitted/data_docs
# files.download('data_docs.zip')

### 7. Using the User Configurable Profiler to compare two tables

The configurable profiler feature provided by Great Expectations can be a great place to start with a never seen before dataset. It creates automated Expectations
based on the values and aggregates of the data. Another way to utilise it is to create automated expectations on one table of data, and then validate them against
a second table, which allows a deep comparison between the two.

In [ ]:
# Exclude most columns so the expectation creation process is not too long
exclude_column_names = ['LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', "LotConfig", "LandSlope", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "MasVnrArea", "ExterQual", "ExterCond", "Foundation", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "Heating", "HeatingQC", "CentralAir", "Electrical", "PoolArea", "PoolQC", "LowQualFinSF", "GrLivArea", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr", "KitchenQual", "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu", "GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea", "GarageQual", "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "Fence", "MiscFeature", "MiscVal", "MoSold", "YrSold", "SaleType", "SaleCondition"]

In [ ]:
house_data.drop(exclude_column_names, axis=1).columns

In [ ]:
runtime_batch_request = RuntimeBatchRequest(
    datasource_name=datasource_name,
    data_connector_name=runtime_data_connector_name,
    data_asset_name=data_asset_name,
    runtime_parameters={"batch_data": house_data.drop(exclude_column_names, axis=1)},
    batch_identifiers={
        batch_identifier_name: "step_3",
    },
)

In [ ]:
# Create a validator instance from the batch request
validator = context.get_validator(batch_request=runtime_batch_request)

In [ ]:
profiler = UserConfigurableProfiler(
    profile_dataset=validator,
    excluded_expectations=None,
    ignored_columns=exclude_column_names,
    not_null_only=False,
    primary_or_compound_key=None,
    semantic_types_dict=None,
    table_expectations_only=False,
    value_set_threshold="MANY",
)
suite = profiler.build_suite()
validator.expectation_suite = suite

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)

In [ ]:
# Create a checkpoint based on the expectation suite built using the profiler on the house_data table, and use it to validate the house_data_test
profiled_validator_checkpoint = "profiled_validator"
checkpoint_config = {
    "name": profiled_validator_checkpoint,
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": {
                "datasource_name": datasource_name,
                "data_connector_name": runtime_data_connector_name,
                "data_asset_name": data_asset_name,
            },
            "expectation_suite_name": "default",
        }
    ],
}
context.add_checkpoint(**checkpoint_config)
results_profiled_checkpoint = context.run_checkpoint(
    checkpoint_name=profiled_validator_checkpoint,
    batch_request={
        "runtime_parameters": {"batch_data": house_data_test.drop(exclude_column_names, axis=1)},
        "batch_identifiers": {
            batch_identifier_name: "step_4"
        },
    },
)

In [ ]:
# Run and open the data docs to look at the comparison summary
context.build_data_docs()

validation_result_identifier = results_profiled_checkpoint.list_validation_result_identifiers()[0]
context.open_data_docs(resource_identifier=validation_result_identifier)

In [ ]:
# in Google Colab you'll have to download the folder and open it up locally in /uncommited/data_docs/local_site/index.html. 
# Uncomment and run the 3 lines below:
# from google.colab import files
# !zip -r data_docs.zip great_expectations/uncommitted/data_docs
# files.download('data_docs.zip')

### 8. Custom Expectations

We will explain the process of creating a custom expectation using an example. A common method to check for outliers is to calculate the Z-score for each value
and signal out any value with a Z-score above 3.

<div>
<img src="https://datasciencelk.com/wp-content/uploads/2020/05/normal-plot-with-sigma.jpg" width="600"/>
<img src="https://www.gstatic.com/education/formulas2/443397389/en/z_score.svg" width=200/>
</div>
<a href="https://datasciencelk.com/normal-distribution-z-scores-standardization-explained/">https://datasciencelk.com/normal-distribution-z-scores-standardization-explained/</a>

Creating a custom expectation requires creating a separate module, we will use a template provided by Great Expectations and make minor adjustments.

In [ ]:
%%writefile expect_column_z_score_lower_than_3.py
"""
This is a template for creating custom ColumnExpectations.
For detailed instructions on how to use it, please see:
    https://docs.greatexpectations.io/docs/guides/expectations/creating_custom_expectations/how_to_create_custom_column_aggregate_expectations
"""

import json
from typing import Callable, Dict, Optional

from numpy import array

from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.execution_engine import (
    PandasExecutionEngine,
    SparkDFExecutionEngine,
    SqlAlchemyExecutionEngine,
)
from great_expectations.execution_engine.execution_engine import (
    ExecutionEngine,
    MetricDomainTypes,
    MetricPartialFunctionTypes,
)
from great_expectations.expectations.expectation import (
    ColumnMapExpectation,
    ExpectationValidationResult,
)
from great_expectations.expectations.metrics import (
    ColumnMapMetricProvider,
    column_condition_partial,
    metric_partial,
)
from great_expectations.expectations.metrics.import_manager import F, sa
from great_expectations.expectations.util import render_evaluation_parameter_string
from great_expectations.render.renderer.renderer import renderer
from great_expectations.render.types import (
    CollapseContent,
    RenderedStringTemplateContent,
)
from great_expectations.render.util import (
    handle_strict_min_max,
    parse_row_condition_string_pandas_engine,
    substitute_none_for_missing,
)
from great_expectations.validator.metric_configuration import MetricConfiguration

    
# This class defines a Metric to support your Expectation.
# For most ColumnMapExpectations, the main business logic for calculation will live in this class.
class ColumnValuesLowerThanZScoreOf3(ColumnMapMetricProvider):

    # This is the id string that will be used to reference your metric.
    condition_metric_name = "column_values.lower_than_z_score_of_3"

    # This method implements the core logic for the PandasExecutionEngine
    @column_condition_partial(engine=PandasExecutionEngine)
    def _pandas(cls, column, **kwargs):
        return abs((abs(column.mean()) - abs(column))/column.std()) < 3


# This class defines the Expectation itself
class ExpectColumnZScoreLowerThan3(ColumnMapExpectation):
    """This expectation takes the input column, calculates the standarad deviation, mean for the entire column and then calculates the 
    Z-score for each value in the column. Any value with a Z-score larger than 3 is considered an outlier. Z-score is defined as: 
    (value-column_mean)/standard_deviation"""

    # These examples will be shown in the public gallery.
    # They will also be executed as unit tests for your Expectation.
    examples = [
        {
            "data": {"x": [1, 2, 3, 4, 5], "y": [-15, 2, 3, 4, 5]},
            "tests": [
                {
                    "title": "basic_positive_test",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {
                        "column": "x",
                    },
                    "out": {"success": True},
                },
                {
                    "title": "basic_negative_test",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {
                        "column": "y",
                    },
                    "out": {"success": False},
                },
            ],
            "test_backends": [
                {
                    "backend": "pandas",
                    "dialects": None,
                },
                # {
                #     "backend": "sqlalchemy",
                #     "dialects": ["sqlite", "postgresql"],
                # },
                # {
                #     "backend": "spark",
                #     "dialects": None,
                # },
            ],
        }
    ]

    # This is the id string of the Metric used by this Expectation.
    # For most Expectations, it will be the same as the `condition_metric_name` defined in your Metric class above.
    map_metric = "column_values.lower_than_z_score_of_3"

    # This is a list of parameter names that can affect whether the Expectation evaluates to True or False
    # Please see https://docs.greatexpectations.io/en/latest/reference/core_concepts/expectations/expectations.html#expectation-concepts-domain-and-success-keys
    # for more information about domain and success keys, and other arguments to Expectations
    success_keys = ("mostly",)

    # This dictionary contains default values for any parameters that should have default values
    default_kwarg_values = {}

    @renderer(renderer_type="renderer.diagnostic.observed_value")
    @render_evaluation_parameter_string
    def _diagnostic_observed_value_renderer(
        cls,
        configuration: ExpectationConfiguration = None,
        result: ExpectationValidationResult = None,
        language: str = None,
        runtime_configuration: dict = None,
        **kwargs,
    ):
        assert result, "Must provide a result object."

        result_dict = result.result
        if result_dict is None:
            return "--"

        if result_dict.get("observed_value"):
            observed_value = result_dict.get("observed_value")
            if isinstance(observed_value, (int, float)) and not isinstance(
                observed_value, bool
            ):
                return num_to_str(observed_value, precision=10, use_locale=True)
            return str(observed_value)
        elif result_dict.get("unexpected_percent") is not None:
            return (
                num_to_str(result_dict.get("unexpected_percent"), precision=5)
                + "% unexpected"
            )
        else:
            return "--"

    @renderer(renderer_type="renderer.diagnostic.unexpected_statement")
    @render_evaluation_parameter_string
    def _diagnostic_unexpected_statement_renderer(
        cls,
        configuration: ExpectationConfiguration = None,
        result: ExpectationValidationResult = None,
        language: str = None,
        runtime_configuration: dict = None,
        **kwargs,
    ):
        assert result, "Must provide a result object."

        success = result.success
        result = result.result

        if result.exception_info["raised_exception"]:
            exception_message_template_str = (
                "\n\n$expectation_type raised an exception:\n$exception_message"
            )

            exception_message = RenderedStringTemplateContent(
                **{
                    "content_block_type": "string_template",
                    "string_template": {
                        "template": exception_message_template_str,
                        "params": {
                            "expectation_type": result.expectation_config.expectation_type,
                            "exception_message": result.exception_info[
                                "exception_message"
                            ],
                        },
                        "tag": "strong",
                        "styling": {
                            "classes": ["text-danger"],
                            "params": {
                                "exception_message": {"tag": "code"},
                                "expectation_type": {
                                    "classes": ["badge", "badge-danger", "mb-2"]
                                },
                            },
                        },
                    },
                }
            )

            exception_traceback_collapse = CollapseContent(
                **{
                    "collapse_toggle_link": "Show exception traceback...",
                    "collapse": [
                        RenderedStringTemplateContent(
                            **{
                                "content_block_type": "string_template",
                                "string_template": {
                                    "template": result.exception_info[
                                        "exception_traceback"
                                    ],
                                    "tag": "code",
                                },
                            }
                        )
                    ],
                }
            )

            return [exception_message, exception_traceback_collapse]

        if success or not result_dict.get("unexpected_count"):
            return []
        else:
            unexpected_count = num_to_str(
                result_dict["unexpected_count"], use_locale=True, precision=20
            )
            unexpected_percent = (
                num_to_str(result_dict["unexpected_percent"], precision=4) + "%"
            )
            element_count = num_to_str(
                result_dict["element_count"], use_locale=True, precision=20
            )

            template_str = (
                "\n\n$unexpected_count unexpected values found. "
                "$unexpected_percent of $element_count total rows."
            )

            return [
                RenderedStringTemplateContent(
                    **{
                        "content_block_type": "string_template",
                        "string_template": {
                            "template": template_str,
                            "params": {
                                "unexpected_count": unexpected_count,
                                "unexpected_percent": unexpected_percent,
                                "element_count": element_count,
                            },
                            "tag": "strong",
                            "styling": {"classes": ["text-danger"]},
                        },
                    }
                )
            ]

    @renderer(renderer_type="renderer.diagnostic.unexpected_table")
    @render_evaluation_parameter_string
    def _diagnostic_unexpected_table_renderer(
        cls,
        configuration: ExpectationConfiguration = None,
        result: ExpectationValidationResult = None,
        language: str = None,
        runtime_configuration: dict = None,
        **kwargs,
    ):
        try:
            result_dict = result.result
        except KeyError:
            return None

        if result_dict is None:
            return None

        if not result_dict.get("partial_unexpected_list") and not result_dict.get(
            "partial_unexpected_counts"
        ):
            return None

        table_rows = []

        if result_dict.get("partial_unexpected_counts"):
            total_count = 0
            for unexpected_count_dict in result_dict.get("partial_unexpected_counts"):
                value = unexpected_count_dict.get("value")
                count = unexpected_count_dict.get("count")
                total_count += count
                if value is not None and value != "":
                    table_rows.append([value, count])
                elif value == "":
                    table_rows.append(["EMPTY", count])
                else:
                    table_rows.append(["null", count])

            if total_count == result_dict.get("unexpected_count"):
                header_row = ["Unexpected Value", "Count"]
            else:
                header_row = ["Sampled Unexpected Values"]
                table_rows = [[row[0]] for row in table_rows]

        else:
            header_row = ["Sampled Unexpected Values"]
            sampled_values_set = set()
            for unexpected_value in result_dict.get("partial_unexpected_list"):
                if unexpected_value:
                    string_unexpected_value = str(unexpected_value)
                elif unexpected_value == "":
                    string_unexpected_value = "EMPTY"
                else:
                    string_unexpected_value = "null"
                if string_unexpected_value not in sampled_values_set:
                    table_rows.append([unexpected_value])
                    sampled_values_set.add(string_unexpected_value)

        unexpected_table_content_block = RenderedTableContent(
            **{
                "content_block_type": "table",
                "table": table_rows,
                "header_row": header_row,
                "styling": {
                    "body": {"classes": ["table-bordered", "table-sm", "mt-3"]}
                },
            }
        )

        return unexpected_table_content_block

    # This dictionary contains metadata for display in the public gallery
    library_metadata = {
        "tags": [],
        "contributors": ["@NatanMish"],
    }


if __name__ == "__main__":
    ExpectColumnZScoreLowerThan3().print_diagnostic_checklist()

In [ ]:
# Running this module as is will run a series of checks to see whether the expectation can be used.
!python expect_column_z_score_lower_than_3.py

In [ ]:
from expect_column_z_score_lower_than_3 import ExpectColumnZScoreLowerThan3

In [ ]:
validator.expect_column_z_score_lower_than3(column="SalePrice")

In [ ]:
validator.expect_column_value_z_scores_to_be_less_than("SalePrice", 3, double_sided=True)

### Exercise 2

In this exercise we will create a custom expectation using the ColumnPairMapExpectation template. This template is useful for creating an expectation that compares values from two columns. The expectation we are going to create is "Expect Proportional Floor SF(square feet) Ratio". This expectation will pick up two columns - `1stFlrSF` and `2ndFlrSF` and validate that the 2nd floor is not more than twice larger than the first one.

To build this custom expectation follow these steps:
1. In line 37, insert a Pandas expression that returns a Series of booleans: True if the expectation holds, meaning the ratio between the 2nd floor SF to the 1st one is less than 2, and False otherwise.
2. In lines 48-49, insert values for the two lists that are going to be used in the tests. col_a is for the 1st floor SF and col_b is for the 2nd floor SF. Notice there is one positive scenario and one negative scenario. The `mostly` success key is set at 0.6 for the succesful scenario and 1 for the failed scenario, which means that in your lists at least 60% of the pairs should be valid according to the logic. For example, two valid pairs and one non-valid.
3. Run the rest of the cells and see that checks are passing and the results you get are as expected.

To add line numbers in Google Colab select Tools -> Settings -> Editor -> show line numbers
To add line numbers in Jupyter select view -> toggle line numbers

**For the purpose of this exercise, column A is the 1st flor SF and column B is the 2nd floor SF.**

In [ ]:
%%writefile expect_proportional_floor_sf_ratio.py
from typing import Dict, Optional

from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.exceptions.exceptions import (
    InvalidExpectationConfigurationError,
)
from great_expectations.execution_engine import (
    ExecutionEngine,
    PandasExecutionEngine,
    SparkDFExecutionEngine,
    SqlAlchemyExecutionEngine,
)
from great_expectations.expectations.expectation import (
    ColumnPairMapExpectation,
    ExpectationValidationResult,
)
from great_expectations.expectations.metrics.import_manager import F, sa
from great_expectations.expectations.metrics.map_metric_provider import (
    ColumnPairMapMetricProvider,
    column_pair_condition_partial,
)
from great_expectations.validator.metric_configuration import MetricConfiguration


class ColumnFloorsSquareFeetComparison(ColumnPairMapMetricProvider):
    """MetricProvider Class for columns floors square feet comparison"""
    condition_metric_name = "column_pair_values.floors_square_feet_ratio"
    condition_domain_keys = (
        "column_A",
        "column_B",
    )
    condition_value_keys = ()
    @column_pair_condition_partial(engine=PandasExecutionEngine)
    def _pandas(cls, column_A, column_B, **kwargs):
        # This methold should return a Pandas series of booleans
        return <YOUR PANDAS EXPRESSION HERE>


class ExpectProportionalFloorDifference(ColumnPairMapExpectation):
    """Expect house 2nd floor to be no more than twice larger than the 1st floor"""
    map_metric = "column_pair_values.floors_square_feet_ratio"
    # These examples will be shown in the public gallery.
    # They will also be executed as unit tests for your Expectation.
    examples = [
        {
            "data": {
                "col_a": [<test values for 1st floor SF>],
                "col_b": [<test values for 2nd floor SF>],
            },
            "tests": [
                {
                    "title": "basic_positive_test",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column_A": "col_a", "column_B": "col_b", "mostly": 0.6},
                    "out": {
                        "success": True,
                    },
                },
                {
                    "title": "basic_negative_test",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column_A": "col_a", "column_B": "col_b", "mostly": 1},
                    "out": {
                        "success": False,
                    },
                },
            ],
        }
    ]
    # Setting necessary computation metric dependencies and defining kwargs, as well as assigning kwargs default values
    success_keys = (
        "column_A",
        "column_B",
        "mostly",
    )

    default_kwarg_values = {
        "row_condition": None,
        "condition_parser": None,  # we expect this to be explicitly set whenever a row_condition is passed
        "mostly": 1.0,
        "result_format": "COMPLETE",
        "include_config": True,
        "catch_exceptions": False,
    }
    args_keys = (
        "column_A",
        "column_B",
    )

    def validate_configuration(
        self, configuration: Optional[ExpectationConfiguration]
    ) -> None:
        super().validate_configuration(configuration)
        if configuration is None:
            configuration = self.configuration
        try:
            assert (
                "column_A" in configuration.kwargs
                and "column_B" in configuration.kwargs
            ), "both columns must be provided"
        except AssertionError as e:
            raise InvalidExpectationConfigurationError(str(e))

    # This dictionary contains metadata for display in the public gallery
    library_metadata = {
        "tags": [],
        "contributors": ["<YOUR GITHUB USERNAME HERE>"],
    }


if __name__ == "__main__":
    ExpectProportionalFloorDifference().print_diagnostic_checklist()
# Note to users: code below this line is only for integration testing -- ignore!

diagnostics = ExpectProportionalFloorDifference().run_diagnostics()

for check in diagnostics["tests"]:
    assert check["test_passed"] is True
    assert check["error_diagnostics"] is None

for check in diagnostics["errors"]:
    assert check is None

for check in diagnostics["maturity_checklist"]["experimental"]:
    if check["message"] == "Passes all linting checks":
        continue
    assert check["passed"] is True

In [ ]:
# Check that the module was configured correctly
!python expect_proportional_floor_sf_ratio.py

In [ ]:
from expect_proportional_floor_sf_ratio import ExpectProportionalFloorDifference

In [ ]:
# Run the expectation using the validator instance, you should get one unexpected value in the results
validator.expect_proportional_floor_difference("1stFlrSF", "2ndFlrSF")

*Exercise solutions can be found in the exercise solutions file in the current directory.*